# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
!pip install elasticsearch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 381 kB 8.1 MB/s 
     |████████████████████████████████| 59 kB 5.6 MB/s 
     |████████████████████████████████| 138 kB 58.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [4]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [5]:
import json
dataset_path = "/content/drive/MyDrive/IR_P3/IR_data_news_12k.json"
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])


## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [6]:
import random
index_name = f"{random.randint(1, 1000)}_th_index"
ELASTIC_PASSWORD = "6RcaBqaCzfoPaoERT95g1Jwl"

# Found in the 'Manage Deployment' page
CLOUD_ID = "deployment-name:dXMtZWFzdDQuZ2Nw..."

# Create the client instance
es = Elasticsearch(
    cloud_id="th3amirrj:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvOjQ0MyQzOTBjODA5YjIwOGI0NGI3OGZlNWI4Y2JhMjhiMzViOSRjZjUxOGIyYmQ4OTk0NDJmOGJlNzBhZTIxOTY0ZDQ5NA==",
    basic_auth=("elastic", ELASTIC_PASSWORD)
)
es.indices.create(index=index_name)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': '983_th_index'})

### Check the Cluster Status

In [7]:
dict(es.info())

{'cluster_name': '390c809b208b44b78fe5b8cba28b35b9',
 'cluster_uuid': 'YNINsOPFRDyYB2NnI68A0Q',
 'name': 'instance-0000000003',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2022-06-23T22:48:49.607492124Z',
  'build_hash': '5b8b981647acdf1ba1d88751646b49d1b461b4cc',
  'build_snapshot': False,
  'build_type': 'docker',
  'lucene_version': '9.2.0',
  'minimum_index_compatibility_version': '7.0.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'number': '8.3.0'}}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [ ]:
for i in tqdm(range(len(data))):
    es.index(index = index_name, id=i, document=data[str(i)])
    i = i + 1

100%|██████████| 12202/12202 [01:30<00:00, 134.28it/s]


###  Bulk API

In [8]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': index_name,
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [9]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 26.68 seconds


### check index

In [10]:
es.count(index=index_name)

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [11]:
query1 = {
        "bool": {
          "should": [
              { 
                  #TODO: add a match query structure ==> use for normal words
                  "match": {"content": "آمریکا"}
              }, 
              
              { 
                  #TODO: add a match pharse query strucutre ==> use for words in <"">
                  "match_phrase": {"content": "تحریم هسته‌ای"}
              },
   
          ],
          "must_not": [
              {
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
                  "match": {"content": "ایران"}
              }
          ],
        },
    }

In [12]:
query2 = {
        "bool": {
          "should": [
              { 
                  #TODO: add a match query structure ==> use for normal words
                  "match": {"content": "اورشلیم"}
              }
   
          ],
          "must_not": [
              {
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
                  "match": {"content": "صهیونیست"}
              }
          ],
        },
    }

### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [13]:
res = es.search(index = index_name, query=query1, _source= ["url"])
res = dict(res)

### Results

In [14]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took'] / 1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

171 results in 1.267 s: 
https://www.farsnews.ir/news/14001214000789/نماینده-کلیمیان-در-مجلس-دهم--کارتل‌ها-بر-تصمیمات-هیأت-حاکمه-آمریکا
https://www.farsnews.ir/news/14001217000665/خباثت‌های-آمریکا-در-برجام-روسیه-و-چین-را-هم-به-این-کشور-بدبین-کرد
https://www.farsnews.ir/news/14001213000089/آمریکا-با-بحران‌آفرینی-به-حیات-خود-ادامه-می-دهد
https://www.farsnews.ir/news/14001211000321/آمریکا-دولت‌های-متحد-خود-را-در-زمان-اضطرار-تنها-می‌گذارد
https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-آمریکا-در-ناامن-بودن-جهان-است
https://www.farsnews.ir/news/14001214000825/آمریکا-رژیمی-مافیایی-است-و-مردم-در-تصمیمات-حاکمان-آن-جایگاهی-ندارند
https://www.farsnews.ir/news/14001212000725/آمریکا-برای-فروش-تسلیحات-خود-به-ایجادناامنی-و-بحران‌آفرینی-نیاز-دارد
https://www.farsnews.ir/news/14001204000444/۲۸-آمریکایی-در-جام-یاشاردوغو-جردن-باروز-هم-می‌آید
https://www.farsnews.ir/news/14001213000328/کارتل‌های-اقتصادی-رؤسای-جمهور-آمریکا-را-تعیین-می‌کنند
https://www.farsnews.ir/news/14001211000220/نمای

In [15]:
res = es.search(index = index_name, query=query2, _source= ["url"])
res = dict(res)

In [16]:
print("{} results in {} s: ".format(res['hits']['total']['value'] ,res['took'] / 1000))
for doc in res['hits']['hits']:
    print(doc['_source']['url'])

0 results in 0.001 s: 
